In [32]:
import pandas as pd
import os
from dotenv import load_dotenv, find_dotenv
import unidecode
from lyricsgenius import Genius
import spotipy
import spotipy.util as util
import text2emotion as te
from difflib import SequenceMatcher

In [30]:
env_path = find_dotenv()
load_dotenv(env_path)

True

In [3]:
data = pd.read_csv('../raw_data/data.csv')
data.head()

,valence,year,acousticness,artists,danceability,duration_ms,energy,explicit,id,instrumentalness,key,liveness,loudness,mode,name,popularity,release_date,speechiness,tempo
0,0.0594,1921,0.982,"['Sergei Rachmaninoff', 'James Levine', 'Berli...",0.279,831667,0.211,0,4BJqT0PrAfrxzMOxytFOIz,0.878000,10,0.665,-20.096,1,"Piano Concerto No. 3 in D Minor, Op. 30: III. ...",4,1921,0.0366,80.954
1,0.9630,1921,0.732,['Dennis Day'],0.819,180533,0.341,0,7xPhfUan2yNtyFG0cUWkt8,0.000000,7,0.160,-12.441,1,Clancy Lowered the Boom,5,1921,0.4150,60.936
2,0.0394,1921,0.961,['KHP Kridhamardawa Karaton Ngayogyakarta Hadi...,0.328,500062,0.166,0,1o6I8BglA6ylDMrIELygv1,0.913000,3,0.101,-14.850,1,Gati Bali,5,1921,0.0339,110.339
3,0.1650,1921,0.967,['Frank Parker'],0.275,210000,0.309,0,3ftBPsC5vPBKxYSee08FDH,0.000028,5,0.381,-9.316,1,Danny Boy,3,1921,0.0354,100.109
4,0.2530,1921,0.957,['Phil Regan'],0.418,166693,0.193,0,4d6HGyGT8e121BsdKmw9v6,0.000002,3,0.229,-10.096,1,When Irish Eyes Are Smiling,2,1921,0.0380,101.665


In [24]:
data_sample = data.sample(100, ignore_index=True)
data_sample.head()

,valence,year,acousticness,artists,danceability,duration_ms,energy,explicit,id,instrumentalness,key,liveness,loudness,mode,name,popularity,release_date,speechiness,tempo
0,0.441,1984,0.637,['Everything But The Girl'],0.411,168467,0.712,0,5QWexGWc05oS95sUJ4braI,0.00000,8,0.0622,-6.377,0,Each and Every One - 2012 Remaster,35,1984-06-04,0.0449,142.991
1,0.574,1968,0.690,['Engelbert Humperdinck'],0.318,154707,0.553,0,51OV8BtjCOPxTVLrbHBTeN,0.00000,2,0.2980,-8.750,1,"Up, Up And Away",28,1968-08-03,0.0344,164.904
2,0.416,1944,0.989,"['Isaac Albéniz', 'Andrés Segovia']",0.514,252933,0.148,0,5AHoDoMfXlciY8JzZLiHvH,0.92300,6,0.1030,-22.739,1,Sevilla,0,1944-01-01,0.0409,121.748
3,0.618,2019,0.220,"['Bastille', 'Alessia Cara']",0.685,213511,0.742,0,2Fn4gZI3MAeWTiv7cSVxZQ,0.00000,8,0.1850,-5.409,1,Another Place,68,2019-11-01,0.0471,115.008
4,0.228,1977,0.932,['Meat Loaf'],0.356,280533,0.363,0,0fAhbZIPf3T1tgX9eDY3Fd,0.00161,7,0.1580,-11.745,1,Heaven Can Wait,48,1977-10-21,0.0297,120.546


In [4]:
artist_test = data['artists'][0].replace('[','').replace(']','').replace("'",'').replace(",",'')
artist_test

'Sergei Rachmaninoff James Levine Berliner Philharmoniker'

In [5]:
song_test = data['name'][0].replace('[','').replace(']','').replace("'",'').replace(",",'')
song_test

'Piano Concerto No. 3 in D Minor Op. 30: III. Finale. Alla breve'

In [28]:
for artist in range(len(data_sample['artists'])):
    
    data_sample['artists'][artist] = data_sample['artists'][artist].replace('[','').replace(']','').replace("'",'').replace(",",'').replace(":",'').replace(")",'').replace("(",'').replace(".",'').replace("-",'')

for name in range(len(data_sample['name'])):

    data_sample['name'][name] = data_sample['name'][name].replace('[','').replace(']','').replace("'",'').replace(",",'').replace(":",'').replace(")",'').replace("(",'').replace(".",'').replace("-",'')

data_sample.head()

/var/folders/7k/g6_9j05d4qg33cskxqfft6dh0000gn/T/ipykernel_8813/3097083059.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_sample['artists'][artist] = data_sample['artists'][artist].replace('[','').replace(']','').replace("'",'').replace(",",'').replace(":",'').replace(")",'').replace("(",'').replace(".",'').replace("-",'')
/var/folders/7k/g6_9j05d4qg33cskxqfft6dh0000gn/T/ipykernel_8813/3097083059.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_sample['name'][name] = data_sample['name'][name].replace('[','').replace(']','').replace("'",'').replace(",",'').replace(":",'').replace(")",'').replace("(",''

,valence,year,acousticness,artists,danceability,duration_ms,energy,explicit,id,instrumentalness,key,liveness,loudness,mode,name,popularity,release_date,speechiness,tempo
0,0.441,1984,0.637,Everything But The Girl,0.411,168467,0.712,0,5QWexGWc05oS95sUJ4braI,0.00000,8,0.0622,-6.377,0,Each and Every One 2012 Remaster,35,1984-06-04,0.0449,142.991
1,0.574,1968,0.690,Engelbert Humperdinck,0.318,154707,0.553,0,51OV8BtjCOPxTVLrbHBTeN,0.00000,2,0.2980,-8.750,1,Up Up And Away,28,1968-08-03,0.0344,164.904
2,0.416,1944,0.989,Isaac Albéniz Andrés Segovia,0.514,252933,0.148,0,5AHoDoMfXlciY8JzZLiHvH,0.92300,6,0.1030,-22.739,1,Sevilla,0,1944-01-01,0.0409,121.748
3,0.618,2019,0.220,Bastille Alessia Cara,0.685,213511,0.742,0,2Fn4gZI3MAeWTiv7cSVxZQ,0.00000,8,0.1850,-5.409,1,Another Place,68,2019-11-01,0.0471,115.008
4,0.228,1977,0.932,Meat Loaf,0.356,280533,0.363,0,0fAhbZIPf3T1tgX9eDY3Fd,0.00161,7,0.1580,-11.745,1,Heaven Can Wait,48,1977-10-21,0.0297,120.546


In [33]:
def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()

In [48]:
# loading needed credentials
genius_token = os.environ.get('LYRICSGENIUS_ACCESS_TOKEN')

# instantiate the class with some parameters
genius = Genius(genius_token, 
                timeout=120, 
                remove_section_headers=True, 
                sleep_time=0.3, 
                skip_non_songs=True)

for i in range(len(data_sample['name'])):

    title = unidecode.unidecode(data_sample.loc[i]['name'].lower())
    artist = unidecode.unidecode(data_sample.loc[i]['artists'].lower())

    song = genius.search_song(title=title, artist=artist)
    
    if song != None:

        api_response = unidecode.unidecode(song.to_dict()['full_title'].replace('\xa0', ' ').replace('\u200b', ' ').lower())
        api_response = api_response.split(' by ')

        song_similar = similar(api_response[0], title)
        artist_similar = similar(api_response[1], artist)
    
    if song_similar > 0.8:
    
        # all lyrics responses come with the song's title and 'Lyrics' str
        characters_to_remove = len(song.to_dict()['title'] + ' Lyrics')   
        
        # get lyrics
        lyrics = genius.lyrics(song.to_dict()['id'])[characters_to_remove:-5]
    
    else:
        
        lyrics = None
        
    data_sample['lyrics'][i] = lyrics
    
data_sample

Searching for "each and every one  2012 remaster" by everything but the girl...
Done.


KeyError: 'lyrics'

In [47]:
genius.lyrics(song.to_dict()['id'])

"Thanatoid LyricsI hear the voices around me\nBurning the regions of mind\nThe thoughts are growling together\nI gaze at the walls of frustration\nBewitching my last sanity\nSadistic death takes my fear of death\nThanatoid\nMy chance to arrest this torment\nIs vanished by passing of time\nI get confused in a austral disease\nThe deadly complex is pounding\nAnd twisting the walls of my heart\nMy mind is crossing up the threshold...\n... I die\nThanatoid\nNerves on edge, paranoia's near\nExplain the growing sorrow... years by years\nI see my victims turning... from the ways of the past\nThey release my days... of disgust\nSenses, searching for life\nConceiving breath after breath\nDrastic measure to recharge my brain\nI watch in splendour at the mass\nOld dreams, (of) paranormal life\nPsychoneuroses deny\nSeeds of anguish, unfold their roots\nInhaled desire, I want... to pass... away\nStop my human wasteYou might also likeEmbed"

In [35]:
data_sample.loc[2]['artists']

'Isaac Albéniz Andrés Segovia'